In [1]:
# import necessary libraries
import pandas as pd

In [68]:
# daily weather database for the 1st French department
gz_file_path = "data/meteo-quot-01-2023-2024_RR-T-Vent.csv.gz"
df_meteo_quot_01 = pd.read_csv(gz_file_path, compression='gzip', sep=';', encoding='utf-8')

In [69]:
df_meteo_quot_01.sample(10)

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMMJJ,RR,QRR,TN,QTN,...,HXI2,QHXI2,FXI3S,QFXI3S,DXI3S,QDXI3S,HXI3S,QHXI3S,DRR,QDRR
5225,1143002,DIVONNE ZA,46.341500,6.139667,467,20230313,13.1,1.0,3.9,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11246,1414001,SUTRIEU,45.916167,5.624667,878,20240713,0.2,1.0,5.0,1.0,...,NaN,NaN,8.3,1.0,NaN,NaN,1208.0,9.0,NaN,NaN
3841,1072001,CEYZERIAT_SAPC,46.204333,5.287667,260,20230319,9.0,1.0,7.9,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994,1034004,BELLEY,45.769333,5.688000,330,20231112,11.8,1.0,3.6,1.0,...,NaN,NaN,5.0,9.0,NaN,NaN,952.0,9.0,NaN,NaN
9620,1367002,SAINT JULIEN SUR REYSSOUZE,46.403167,5.109833,189,20231118,0.4,1.0,2.8,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8048,1247003,MIJOUX,46.378833,6.019333,1002,20230520,0.0,1.0,6.8,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
460,1014002,ARBENT,46.278167,5.669000,534,20240405,0.0,1.0,9.2,1.0,...,NaN,NaN,12.8,1.0,NaN,NaN,734.0,9.0,NaN,NaN
949,1027003,BALAN_AERO,45.833000,5.106667,196,20231001,NaN,NaN,7.2,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9161,1269001,NANTUA,46.148167,5.607333,483,20240711,16.8,1.0,15.8,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8765,1269001,NANTUA,46.148167,5.607333,483,20230611,5.0,1.0,14.8,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
df_meteo_quot_01.columns

Index(['NUM_POSTE', 'NOM_USUEL', 'LAT', 'LON', 'ALTI', 'AAAAMMJJ', 'RR', 'QRR',
       'TN', 'QTN', 'HTN', 'QHTN', 'TX', 'QTX', 'HTX', 'QHTX', 'TM', 'QTM',
       'TNTXM', 'QTNTXM', 'TAMPLI', 'QTAMPLI', 'TNSOL', 'QTNSOL', 'TN50',
       'QTN50', 'DG', 'QDG', 'FFM', 'QFFM', 'FF2M', 'QFF2M', 'FXY', 'QFXY',
       'DXY', 'QDXY', 'HXY', 'QHXY', 'FXI', 'QFXI', 'DXI', 'QDXI', 'HXI',
       'QHXI', 'FXI2', 'QFXI2', 'DXI2', 'QDXI2', 'HXI2', 'QHXI2', 'FXI3S',
       'QFXI3S', 'DXI3S', 'QDXI3S', 'HXI3S', 'QHXI3S', 'DRR', 'QDRR'],
      dtype='object')

In [71]:
# hourly weather for Paris department
gz_file_path2 = "data/meteo-horaire-75-2023-2024.csv.gz"
df_meteo_hor_75 = pd.read_csv(gz_file_path2, compression='gzip', sep=';', encoding='utf-8')

In [72]:
df_meteo_hor_75.sample(10)

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMMJJHH,RR1,QRR1,DRR1,QDRR1,...,INS,QINS,INS2,QINS2,TLAGON,QTLAGON,TVEGETAUX,QTVEGETAUX,ECOULEMENT,QECOULEMENT
64894,75114001,PARIS-MONTSOURIS,48.821667,2.337833,75,2024090712,0.0,1.0,15.0,9.0,...,20.0,9.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN
32821,75107005,TOUR EIFFEL,48.858333,2.294500,330,2024110209,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39416,75110001,LARIBOISIERE,48.882833,2.352000,55,2023090802,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96821,75116008,LONGCHAMP,48.854833,2.233667,27,2024070712,0.0,1.0,NaN,NaN,...,7.0,9.0,7.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN
90694,75116008,LONGCHAMP,48.854833,2.233667,27,2023102605,0.0,1.0,NaN,NaN,...,0.0,9.0,0.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN
27639,75107005,TOUR EIFFEL,48.858333,2.294500,330,2024033111,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76650,75114007,PARIS-MONTSOURIS-DOUBLE,48.821667,2.337833,75,2024021404,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34000,75110001,LARIBOISIERE,48.882833,2.352000,55,2023012510,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15276,75106001,LUXEMBOURG,48.844667,2.333833,46,2024092812,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92261,75116008,LONGCHAMP,48.854833,2.233667,27,2023123012,0.0,1.0,NaN,NaN,...,0.0,9.0,4.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
df_meteo_hor_75.columns

Index(['NUM_POSTE', 'NOM_USUEL', 'LAT', 'LON', 'ALTI', 'AAAAMMJJHH', 'RR1',
       'QRR1', 'DRR1', 'QDRR1',
       ...
       'INS', 'QINS', 'INS2', 'QINS2', 'TLAGON', 'QTLAGON', 'TVEGETAUX',
       'QTVEGETAUX', 'ECOULEMENT', 'QECOULEMENT'],
      dtype='object', length=204)

In [74]:
columns_to_keep = ["NUM_POSTE", "NOM_USUEL", "LAT", "LON", "AAAAMMJJHH","RR1", "DRR1", "T", "U", "PSTAT"]
df_meteo_hor_75_cleaned = df_meteo_hor_75[columns_to_keep]

In [75]:
df_meteo_hor_75_cleaned

,NUM_POSTE,NOM_USUEL,LAT,LON,AAAAMMJJHH,RR1,DRR1,T,U,PSTAT
0,75106001,LUXEMBOURG,48.844667,2.333833,2023010100,0.0,NaN,15.4,NaN,NaN
1,75106001,LUXEMBOURG,48.844667,2.333833,2023010101,0.0,NaN,15.3,NaN,NaN
2,75106001,LUXEMBOURG,48.844667,2.333833,2023010102,0.0,NaN,15.0,NaN,NaN
3,75106001,LUXEMBOURG,48.844667,2.333833,2023010103,0.0,NaN,14.7,NaN,NaN
4,75106001,LUXEMBOURG,48.844667,2.333833,2023010104,0.0,NaN,14.3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
100240,75116008,LONGCHAMP,48.854833,2.233667,2024112623,0.0,NaN,8.3,97.0,NaN
100241,75116008,LONGCHAMP,48.854833,2.233667,2024112700,0.0,NaN,9.7,95.0,NaN
100242,75116008,LONGCHAMP,48.854833,2.233667,2024112701,0.0,NaN,10.2,93.0,NaN
100243,75116008,LONGCHAMP,48.854833,2.233667,2024112702,0.0,NaN,10.3,94.0,NaN


In [76]:
df_meteo_hor_75_cleaned['NOM_USUEL'].unique()

array(['LUXEMBOURG', 'TOUR EIFFEL', 'LARIBOISIERE', 'PARIS-MONTSOURIS',
       'PARIS-MONTSOURIS-DOUBLE', 'LONGCHAMP'], dtype=object)

In [42]:
# hourly weather for Boulogne department
gz_file_path3 = "data/meteo-horaire-92-2023-2024.csv.gz"
df_meteo_hor_92 = pd.read_csv(gz_file_path3, compression='gzip', sep=';', encoding='utf-8')

In [ ]:
df_meteo_hor_92_cleaned = df_meteo_hor_92[columns_to_keep]

In [ ]:
df_meteo_hor_92_cleaned

In [54]:
df_meteo_hor_92_cleaned['NOM_USUEL'].unique()

array(['SURESNES'], dtype=object)

In [80]:
import requests
from io import BytesIO
import gzip

# Function to download and read CSV from a URL
def download_and_read_csv(url):
    response = requests.get(url)
    if response.status_code == 200:
        # Decompress the content
        with gzip.GzipFile(fileobj=BytesIO(response.content), mode='rb') as f:
            # Read CSV directly from the decompressed content
            df = pd.read_csv(f)
            return df
    else:
        print(f"Failed to download {url}. Status code: {response.status_code}")
        return None

# Base URL for the department weather data
base_url = "https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/HOR/H_"

# List of department numbers (01 to 95)
department_numbers = [f"{i:02d}" for i in range(1, 96)]

# Placeholder for all DataFrames
all_dataframes = []

# Iterate over each department and download data
for dept in department_numbers:
    url = f"{base_url}{dept}_latest-2023-2024.csv.gz"
    print(f"Downloading data for department {dept}...")
    df = download_and_read_csv(url)
    if df is not None:
        # Optionally, add a new column for department number
        df['Department'] = dept
        all_dataframes.append(df)

# Combine all DataFrames into one DataFrame
if all_dataframes:
    final_df = pd.concat(all_dataframes, ignore_index=True)
    print("Data for all departments successfully combined.")
else:
    final_df = pd.DataFrame()
    print("No data was downloaded.")

# Display the first few rows of the combined DataFrame
print(final_df.head())

Data for all departments successfully combined.
  NUM_POSTE;NOM_USUEL;LAT;LON;ALTI;AAAAMMJJHH;RR1;QRR1;DRR1;QDRR1;FF;QFF;DD;QDD;FXY;QFXY;DXY;QDXY;HXY;QHXY;FXI;QFXI;DXI;QDXI;HXI;QHXI;FF2;QFF2;DD2;QDD2;FXI2;QFXI2;DXI2;QDXI2;HXI2;QHXI2;FXI3S;QFXI3S;DXI3S;QDXI3S;HFXI3S;QHFXI3S;T;QT;TD;QTD;TN;QTN;HTN;QHTN;TX;QTX;HTX;QHTX;DG;QDG;T10;QT10;T20;QT20;T50;QT50;T100;QT100;TNSOL;QTNSOL;TN50;QTN50;TCHAUSSEE;QTCHAUSSEE;DHUMEC;QDHUMEC;U;QU;UN;QUN;HUN;QHUN;UX;QUX;HUX;QHUX;DHUMI40;QDHUMI40;DHUMI80;QDHUMI80;TSV;QTSV;PMER;QPMER;PSTAT;QPSTAT;PMERMIN;QPMERMIN;GEOP;QGEOP;N;QN;NBAS;QNBAS;CL;QCL;CM;QCM;CH;QCH;N1;QN1;C1;QC1;B1;QB1;N2;QN2;C2;QC2;B2;QB2;N3;QN3;C3;QC3;B3;QB3;N4;QN4;C4;QC4;B4;QB4;VV;QVV;DVV200;QDVV200;WW;QWW;W1;QW1;W2;QW2;SOL;QSOL;SOLNG;QSOLNG;TMER;QTMER;VVMER;QVVMER;ETATMER;QETATMER;DIRHOULE;QDIRHOULE;HVAGUE;QHVAGUE;PVAGUE;QPVAGUE;HNEIGEF;QHNEIGEF;NEIGETOT;QNEIGETOT;TSNEIGE;QTSNEIGE;TUBENEIGE;QTUBENEIGE;HNEIGEFI3;QHNEIGEFI3;HNEIGEFI1;QHNEIGEFI1;ESNEIGE;QESNEIGE;CHARGENEIGE;QCHARGENEIGE;GLO;QGLO;GL

In [86]:
# Définir la taille des morceaux (par exemple, 10000 lignes)
chunksize = 10000
file_path = 'nom_du_fichier.xlsx'

# Lire le fichier en morceaux
for chunk in pd.read_excel(file_path, chunksize=chunksize, engine='openpyxl'):
    # Afficher les premières lignes de chaque morceau
    print(chunk.head())

TypeError: read_excel() got an unexpected keyword argument 'chunksize'

In [85]:
print('hello world!')

hello world!
